In [60]:
import sys 
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Clearvale/jake_scripts/Amy_flywheel_scripts/')

import numpy as np
import pandas as pd
import os

In [61]:
z_data = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/ClearMem_Z_Average.csv')
z_data = z_data[['SubID', 'z_ave', 'PSWQ_total', 'WBSI_total', 'RRS_total', 'RRS_depression', 'RRS_brooding', 'RRS_reflection']]
z_data = z_data.dropna()

from scipy.stats import zscore
z_data['br_z_ave'] = z_data['z_ave']
z_data.drop('z_ave', axis=1, inplace=True)
z_data['b_z_ave'] = (zscore(z_data['PSWQ_total']) + zscore(z_data['WBSI_total']) + zscore(z_data['RRS_brooding']))/3
z_data['thought_problems'] = (zscore(z_data['PSWQ_total']) + zscore(z_data['WBSI_total']) + zscore(z_data['RRS_brooding']) + zscore(z_data['RRS_reflection']) + zscore(z_data['RRS_depression']))/5

#all_metrics_z= z_data.drop(['PSWQ_total', 'WBSI_total','RRS_total', 'RRS_depression', 
#                     'RRS_brooding', 'RRS_reflection', 'br_z_ave'], axis=1)

In [62]:
op_mat_sub_ids = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/matched_subid.csv')

In [63]:
sub_rsa_mat_cors = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/bootstrapped_regressions/sub_rsa_mat_cors.csv')   
sub_rsa_mat_cors = pd.merge(op_mat_sub_ids, sub_rsa_mat_cors, on='sub').drop('sub', axis=1)

In [64]:
sub_rsa_mat_cors_z = pd.merge(z_data[['SubID', 'thought_problems']], sub_rsa_mat_cors, on='SubID')

In [65]:
#sub_rsa_mat_cors_z 

In [66]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools

class RegressionAnalysis:
    def __init__(self, data, variable_list, target='thought_problems', interaction=None):
        self.data = data
        self.variable_list = variable_list
        self.target = target
        self.interaction = interaction
        self.regression_results = {}
        self.regression_models = {}

    def run_regression(self, target, y_vars):
        if self.interaction is not None:
            if len(y_vars) > 1:
                joined_vars = ' * '.join(y_vars)
            else:
                joined_vars = y_vars[0]
            formula = f'{target} ~ {joined_vars}'
        else:
            joined_vars = ' + '.join(y_vars)
            formula = f'{target} ~ {joined_vars}'

        model = smf.ols(formula=formula, data=self.data).fit()
        summary = model.summary()
        var = pd.DataFrame(summary.tables[0].data).iloc[0, 1]

        table1 = pd.DataFrame(summary.tables[0].data).iloc[:4, 3:].T.assign(var=var)
        table1.columns = ['r2', 'adjr2', 'fstat', 'pval', 'var']
        table1 = table1[['var', 'r2', 'adjr2', 'fstat', 'pval']]

        table2_cols = pd.DataFrame(summary.tables[1].data).loc[0].to_list() + ['var']
        table2_cols[0] = 'parameter'
        table2 = pd.DataFrame(summary.tables[1].data).iloc[1:].assign(var=var)
        table2.columns = table2_cols
        table2 = table2[['var', 'parameter', 'coef', 'std err', 't', 'P>|t|']]

        df = pd.merge(table1, table2, how='outer', left_on='var', right_on='parameter')
        df['var'] = df['var_x'].fillna(df['var_y'])
        df.drop(columns=['var_x', 'var_y'], inplace=True)
        df = df[['var'] + [col for col in df.columns if col != 'var']]
        df = df.assign(formula=formula)

        return df, model

    def run_all_regressions(self):
        for metric in self.variable_list:
            df, model = self.run_regression(self.target, [metric])
            self.regression_results[metric] = df
            self.regression_models[metric] = model

    def get_combined_results(self):
        combined_results = []
        for metric in self.regression_results:
            combined_results.append(self.regression_results[metric])

        output_regressions = pd.concat(combined_results, ignore_index=True)
        columns_to_convert = ['r2', 'adjr2', 'fstat', 'pval', 'coef', 'std err', 't', 'P>|t|']
        for column in columns_to_convert:
            output_regressions[column] = pd.to_numeric(output_regressions[column], errors='coerce')

        return output_regressions
    
def clean_formulas(dataframe, pval_threshold=0.05, target_prefix='thought_problems ~ '):
    """
    Filters formulas in a DataFrame based on p-value and removes the target prefix from each formula.

    Parameters:
    dataframe (pd.DataFrame): The DataFrame containing the regression results.
    pval_threshold (float): The p-value threshold for filtering formulas. Default is 0.05.
    target_prefix (str): The prefix to remove from each formula. Default is 'thought_problems ~ '.

    Returns:
    list: A list of cleaned formulas.
    """
    filtered_formulas = dataframe.query('pval < @pval_threshold').formula.to_list()
    cleaned_formulas = [formula.replace(target_prefix, '') for formula in filtered_formulas]
    return cleaned_formulas

In [67]:
def format_og_regs(regressions_table, neural_met):
    
    regressions_table = regressions_table.query('parameter != "Intercept"')
    
    outs = (regressions_table
     .melt(id_vars='formula')
     .dropna()
     .query('value != "thought_problems"')
     .reset_index()
     .pivot(index=['index', 'formula'], columns='variable', values='value')
    ).reset_index().sort_values('formula')

    # Group by formula and aggregate the non-NA values
    agg_funcs = {col: 'first' for col in outs.columns if col != 'formula'}
    collapsed_df = outs.groupby('formula').agg(agg_funcs).reset_index().assign(data_type = neural_met)

    collapsed_df = collapsed_df[['data_type', 'parameter', 'fstat', 'r2', 'adjr2', 'std err', 't', 'pval',]]
    
    return collapsed_df

In [68]:
cor_list = sub_rsa_mat_cors_z.iloc[:, 2:].columns.to_list()

In [69]:
# Usage example
cor_list = sub_rsa_mat_cors_z.iloc[:, 2:].columns.to_list()

cor_analysis = RegressionAnalysis(sub_rsa_mat_cors_z, cor_list)
cor_analysis.run_all_regressions()
cor_output_regressions = cor_analysis.get_combined_results()
cor_regression_models = cor_analysis.regression_models
cor_sig_vars = clean_formulas(cor_output_regressions)

In [72]:
cor_output_regressions 

,var,r2,adjr2,fstat,pval,parameter,coef,std err,t,P>|t|,formula
0,thought_problems,0.005,-0.016,0.249700,0.620,NaN,NaN,NaN,NaN,NaN,thought_problems ~ main_cors
1,thought_problems,NaN,NaN,NaN,NaN,Intercept,0.1077,0.157,0.687,0.495,thought_problems ~ main_cors
2,thought_problems,NaN,NaN,NaN,NaN,main_cors,-2.1964,4.396,-0.500,0.620,thought_problems ~ main_cors
3,thought_problems,0.003,-0.018,0.148600,0.702,NaN,NaN,NaN,NaN,NaN,thought_problems ~ replace_cors
4,thought_problems,NaN,NaN,NaN,NaN,Intercept,0.0965,0.161,0.600,0.551,thought_problems ~ replace_cors
5,thought_problems,NaN,NaN,NaN,NaN,replace_cors,-1.8690,4.848,-0.386,0.702,thought_problems ~ replace_cors
6,thought_problems,0.002,-0.020,0.092190,0.763,NaN,NaN,NaN,NaN,NaN,thought_problems ~ suppress_cors
7,thought_problems,NaN,NaN,NaN,NaN,Intercept,0.0802,0.146,0.551,0.584,thought_problems ~ suppress_cors
8,thought_problems,NaN,NaN,NaN,NaN,suppress_cors,-1.2682,4.177,-0.304,0.763,thought_problems ~ suppress_cors
9,thought_problems,0.008,-0.014,0.366200,0.548,NaN,NaN,NaN,NaN,NaN,thought_problems ~ clear_cors
